In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
from datetime import *
import pygal             # Usefull for french map
from pygal.style import Style
import matplotlib.pyplot as plt
import requests
import json
import csv

%matplotlib inline

## Import Data

In [44]:
df = pd.read_csv('data.csv')
df.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8,26,27,28,30,33,34,43,73) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,...,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement,codeDepartement
0,0,1807254,14,180725400014,O,1972-05-01,NaN,NaN,NaN,2009-10-02T11:01:38,...,1978-05-31,F,NaN,NaN,NaN,NaN,NaN,NaN,N,54
1,1,5410220,15,541022000015,O,NaN,NN,NaN,NaN,NaN,...,1988-03-31,F,NaN,NaN,NaN,NaN,22.02,NAP,N,80
2,2,5410345,10,541034500010,O,NaN,NaN,NaN,NaN,NaN,...,1984-12-25,F,NaN,NaN,NaN,NaN,79.06,NAP,N,80
3,3,5410394,18,541039400018,O,NaN,NN,NaN,NaN,NaN,...,1987-12-01,F,NaN,NaN,NaN,NaN,64.42,NAP,N,80
4,4,5410428,14,541042800014,O,1954-01-01,NN,NaN,NaN,NaN,...,1989-06-11,F,NaN,NaN,NaN,NaN,70.2C,NAF1993,N,80


## Import Excel file to convert code_naf to activity_name (code_naf2.xlsx)

In [3]:
a = pd.read_excel('code_naf2.xlsx')

# 1) Density By Sector of Activity

### Select Data by sector

In [4]:
sector = '55.4B'

In [5]:
df_selection = df[(df['activitePrincipaleEtablissement'] == sector)].groupby('codeDepartement').count()

In [6]:
df_selection.head()

,Unnamed: 0,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,...,dateFin,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
codeDepartement,,,,,,,,,,,,,,,,,,,,,
35,1,1,1,1,1,0,1,0,0,0,...,0,1,1,0,0,0,0,1,1,1
40,11,11,11,11,11,7,8,1,0,4,...,4,11,10,0,0,0,0,11,11,9
41,4,4,4,4,4,4,4,0,0,2,...,2,4,4,0,0,0,0,4,4,4
42,3,3,3,3,3,2,3,0,0,1,...,1,3,3,0,0,0,0,3,3,3
44,40,40,40,40,40,14,36,0,0,8,...,7,40,40,1,0,0,0,40,40,40


### Create French map by departement (Using PYGAL)

#### Custome style from map

In [7]:
custom_style = Style(
  background='transparent',
  transition='200ms ease-in',
)

#### Select activity_name

In [8]:
name_code_naf = a.loc[a['code'] == sector]

In [9]:
name_code_naf = name_code_naf.iloc[0,1]

In [10]:
name_code_naf

'Débits de boissons'

#### Map function

In [11]:
def chart():
    fr_chart = pygal.maps.fr.Departments(style=custom_style,height=450,human_readable=True)    #,legend_at_bottom=True)
    fr_chart.title = 'Density by Sector of Activity'
    fr_chart.add(name_code_naf, {x for x in zip(df_selection.index ,df_selection['siren'])})
    fr_chart.render_in_browser() # Exporte to Browser

#### Print Map

In [12]:
chart()

file:///var/folders/v3/6rms4ljn2dzdln_l8746trf00000gn/T/tmpgmhe8gge.html


# 2) Density of population by Departement

### Import Datas (pop_2019.csv)

In [13]:
df_pop = pd.read_csv('pop_2019.csv')

In [14]:
df_pop.head()

,Unnamed: 0,code,pop
0,0,1,653688
1,1,2,528016
2,2,3,333065
3,3,4,161980
4,4,5,141784


### Create French map by departement (Using PYGAL)

#### Custom stryle from map

In [15]:
custom_style = Style(
  background='transparent',
  transition='200ms ease-in',
)

#### Map function

In [16]:
def chart():
    fr_chart = pygal.maps.fr.Departments(style=custom_style,height=450,human_readable=True)    #,legend_at_bottom=True)
    fr_chart.title = 'Population Density'
    fr_chart.add('In 2019', {x for x in zip(df_pop['code'] ,df_pop['pop'])})
    fr_chart.render_in_browser() # Exporte to Browser

#### Print Map

In [17]:
chart()

file:///var/folders/v3/6rms4ljn2dzdln_l8746trf00000gn/T/tmp9wqkhgyh.html


## 3) Ratio  People/ Companies

### Create Ratio

In [18]:
ratio_list = list()
for val,val2 in zip(df_selection.index,df_selection['siren']):
    for val3,val4 in zip(df_pop['code'] ,df_pop['pop']):
        if val == val3:
            ratio_list.append((val,val2,val4))
            

In [19]:
ratio = list()
for i in range(len(ratio_list)):
    ratio.append((ratio_list[i][0],ratio_list[i][2]/ratio_list[i][1]))

### Create French map by departement (Using PYGAL)

#### Map function

In [20]:
def chart():
    fr_chart = pygal.maps.fr.Departments(style=custom_style,height=450,human_readable=True)    #,legend_at_bottom=True)
    fr_chart.title = 'Ratio Population/compagnie'
    fr_chart.add('Ratio In 2019', {x for x in ratio})
    fr_chart.render_in_browser() # Exporte to Browser

#### Print Map

In [21]:
chart()

file:///var/folders/v3/6rms4ljn2dzdln_l8746trf00000gn/T/tmpdcf7_8ek.html


## 4) Date Close, less 5 years exprerience

### Choose time experience

In [22]:
date_close = 5

In [23]:
df_selection2 = df[(df['activitePrincipaleEtablissement'] == sector)]

### Convert dateDebut and dateFin to Datetime

In [24]:
df_selection2['dateDebut'] = pd.to_datetime(df_selection2['dateDebut'])
df_selection2['dateFin'] = pd.to_datetime(df_selection2['dateFin'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Value dateFin null

In [25]:
df_selection2['dateFin'].isnull().sum()

62

In [26]:
count_close = 0
for i in range(len(df)):
    try:
        if not math.isnan(df_selection2['dateFin'][i].year):
            if df_selection2['dateFin'][i].year <= df_selection2['dateDebut'][i].year + date_close:
                count_close += 1
    except:
        pass

In [27]:
print("Companies close before 5 years experience : ", count_close)

Companies close before 5 years experience :  3


## 5) Distance 50 Km

### Load Data GPS

In [28]:
gps = pd.read_csv('commune_gps.csv',sep=';',error_bad_lines=False)

In [29]:
gps

,EU_circo,code_région,nom_région,chef-lieu_région,numéro_département,nom_département,préfecture,numéro_circonscription,nom_commune,codes_postaux,code_insee,latitude,longitude,éloignement
0,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Attignat,01340,1024,46.283333,5.166667,1.21
1,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Beaupont,01270,1029,46.4,5.266667,1.91
2,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Bény,01370,1038,46.333333,5.283333,1.51
3,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Béreyziat,01340,1040,46.366667,5.05,1.71
4,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Bohas-Meyriat-Rignat,01250,1245,46.133333,5.4,1.01
5,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Bourg-en-Bresse,01000,1053,46.2,5.216667,1
6,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Buellas,01310,1065,46.2,5.133333,0.73
7,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Certines,01240,1069,46.133333,5.266667,1.97
8,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Ceyzériat,01250,1072,46.166667,5.316667,1.15
9,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Chavannes-sur-Suran,01250,1095,46.266667,5.416667,1.14


### Select Siret

In [30]:
siret = 541042800014

### Select Row from Siret

In [31]:
df_gps = df.loc[df['siret'] == siret]
df_gps['activitePrincipaleEtablissement']

4    70.2C
Name: activitePrincipaleEtablissement, dtype: object

### Select Activity

In [32]:
activity = df_gps.iloc[0,77]
activity

'70.2C'

### Select codeCommune

In [33]:
codeCommune = int(df_gps.iloc[0,51])
codeCommune

80001

### Select Latitude

In [34]:
df_latitude_longitude = gps.loc[gps['code_insee'] == codeCommune]
lalitude = df_latitude_longitude.iloc[0,11]
lalitude

'50.1'

### Select Longitude

In [35]:
longitude = df_latitude_longitude.iloc[0,12]
longitude

'1.833333'

### Find Latitude Longitude other Commune

In [36]:
df_selection2 = df[(df['activitePrincipaleEtablissement'] == activity)].groupby('codeCommuneEtablissement').count()
df_selection2.head()

,Unnamed: 0,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,...,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement,codeDepartement
codeCommuneEtablissement,,,,,,,,,,,,,,,,,,,,,
4006.0,3,3,3,3,3,1,3,0,0,1,...,3,3,0,0,0,0,3,3,3,3
4008.0,1,1,1,1,1,0,1,0,0,0,...,1,1,0,0,0,0,1,1,1,1
4019.0,11,11,11,11,11,7,10,1,0,4,...,11,11,1,0,0,0,11,11,10,11
4022.0,3,3,3,3,3,3,3,0,0,1,...,3,3,0,0,0,0,3,3,3,3
4031.0,1,1,1,1,1,1,1,0,0,1,...,1,1,0,0,0,0,1,1,1,1


In [37]:
list_Latitude = list()
list_Longitude = list()
list_codeCommune = list()


for val in df_selection2.index:
    try:
        df_latitude_longitude = gps.loc[gps['code_insee'] == val]
        list_Latitude.append(df_latitude_longitude.iloc[0,11])
        list_Longitude.append(df_latitude_longitude.iloc[0,12])
        list_codeCommune.append(val)
    except:
        pass

### Function Distance

In [38]:
from math import sin, cos, acos, pi
 
#############################################################################
def deg2rad(dd):
    """Convertit un angle "degrés décimaux" en "radians"
    """
    return dd/180*pi
 
#############################################################################
def distanceGPS(latA, longA, latB, longB):
    """Retourne la distance en mètres entre les 2 points A et B connus grâce à
       leurs coordonnées GPS (en radians).
    """
    # Rayon de la terre en mètres (sphère IAG-GRS80)
    RT = 6378137
    # angle en radians entre les 2 points
    S = acos(sin(latA)*sin(latB) + cos(latA)*cos(latB)*cos(abs(longB-longA)))
    # distance entre les 2 points, comptée sur un arc de grand cercle
    return S*RT
 
#############################################################################

list_final_commune = list()
list_final_dist = list()

for i in range(len(list_Latitude)):
    try:
        # cooordonnées GPS en radians du 1er point (ici, mairie de Tours)
        latA = deg2rad(float(lalitude)) # Nord
        longA = deg2rad(float(longitude)) # Est

        # cooordonnées GPS en radians du 2ème point (ici, mairie de Limoges)
        latB = deg2rad(float(list_Latitude[i])) # Nord
        longB = deg2rad(float(list_Longitude[i])) # Est

        dist = distanceGPS(latA, longA, latB, longB)
        #list_final_commune.append(int(dist))
        
        if int(dist)/1000 < 30:
            list_final_commune.append(list_codeCommune[i])
            list_final_dist.append(int(dist))
    except:
        #list_final_commune.append('nan')
        pass

In [39]:
list_final_departement = list()
for val in list_final_commune:
    df_depart = gps.loc[gps['code_insee'] == val]
    list_final_departement.append(df_depart.iloc[0,4])
    #print(list_final_departement[-1])

### Print Commune

In [42]:
def print_concurrence():
    print("Commune à moins de 30 Km :")
    for i in range(len(list_final_commune)):
        print("\t Commune : %s, Distance : %s Km" %(list_final_commune[i],int(list_final_dist[i]/1000)))

In [43]:
print_concurrence()

Commune à moins de 30 Km :
	 Commune : 80001.0, Distance : 0 Km
	 Commune : 80008.0, Distance : 17 Km
	 Commune : 80019.0, Distance : 15 Km
	 Commune : 80063.0, Distance : 24 Km
	 Commune : 80078.0, Distance : 6 Km
	 Commune : 80109.0, Distance : 21 Km
	 Commune : 80126.0, Distance : 23 Km
	 Commune : 80127.0, Distance : 24 Km
	 Commune : 80133.0, Distance : 16 Km
	 Commune : 80148.0, Distance : 21 Km
	 Commune : 80149.0, Distance : 5 Km
	 Commune : 80161.0, Distance : 8 Km
	 Commune : 80190.0, Distance : 13 Km
	 Commune : 80215.0, Distance : 12 Km
	 Commune : 80222.0, Distance : 17 Km
	 Commune : 80228.0, Distance : 20 Km
	 Commune : 80235.0, Distance : 22 Km
	 Commune : 80248.0, Distance : 23 Km
	 Commune : 80262.0, Distance : 5 Km
	 Commune : 80268.0, Distance : 4 Km
	 Commune : 80280.0, Distance : 10 Km
	 Commune : 80308.0, Distance : 17 Km
	 Commune : 80373.0, Distance : 24 Km
	 Commune : 80380.0, Distance : 14 Km
	 Commune : 80396.0, Distance : 19 Km
	 Commune : 80406.0, Distance

### Create French map by departement (Using PYGAL)

In [154]:
def chart():
    save = list()
    fr_chart = pygal.maps.fr.Departments(style=custom_style,height=450)    #,legend_at_bottom=True)
    fr_chart.title = 'Concurrences < 30 Km'
    for x in list_final_departement:
        if x not in save:
            fr_chart.add(x, {x : list_final_departement.count(x)})
            save.append(x)
    fr_chart.render_in_browser() # Exporte to Browser


#### Print Map

In [155]:
chart()

file:///var/folders/v3/6rms4ljn2dzdln_l8746trf00000gn/T/tmpqj3ojr78.html


## 6) TEST

In [46]:
df.head()

,Unnamed: 0,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,...,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement,codeDepartement
0,0,1807254,14,180725400014,O,1972-05-01,NaN,NaN,NaN,2009-10-02T11:01:38,...,1978-05-31,F,NaN,NaN,NaN,NaN,NaN,NaN,N,54
1,1,5410220,15,541022000015,O,NaN,NN,NaN,NaN,NaN,...,1988-03-31,F,NaN,NaN,NaN,NaN,22.02,NAP,N,80
2,2,5410345,10,541034500010,O,NaN,NaN,NaN,NaN,NaN,...,1984-12-25,F,NaN,NaN,NaN,NaN,79.06,NAP,N,80
3,3,5410394,18,541039400018,O,NaN,NN,NaN,NaN,NaN,...,1987-12-01,F,NaN,NaN,NaN,NaN,64.42,NAP,N,80
4,4,5410428,14,541042800014,O,1954-01-01,NN,NaN,NaN,NaN,...,1989-06-11,F,NaN,NaN,NaN,NaN,70.2C,NAF1993,N,80


In [158]:
len(df['dateDebut'])

10000

In [160]:
count = 0
count2 = 0
import math
for i in range(len(df)):
    if df['dateCreationEtablissement'][i] == df['dateDebut'][i]:
        #print(i,"/",df['dateDebut'][i],"/",df['dateCreationEtablissement'][i])
        count += 1
        l_m = 1430
        print(i,"/",df['dateCreationEtablissement'][i],"/",df['dateDebut'][i])
    try:
        if math.isnan(df['dateCreationEtablissement'][i]):
            print(i,"/ nan")
            count2 +=1
    except:
        pass
print("nb_equal : ",count)
print("nb_nan : ",count2)
print("nb_ : ",count+count2)
print("total : ",len(df))


1 / nan
2 / nan
3 / nan
5 / nan
6 / nan
7 / nan
8 / nan
9 / nan
10 / nan
11 / nan
12 / nan
13 / nan
15 / nan
16 / nan
17 / nan
18 / nan
19 / nan
21 / nan
22 / nan
23 / nan
24 / nan
30 / nan
31 / nan
32 / nan
38 / nan
39 / nan
40 / nan
41 / nan
43 / nan
45 / nan
50 / nan
51 / nan
52 / nan
53 / nan
54 / nan
56 / nan
57 / nan
58 / nan
59 / nan
60 / nan
61 / nan
62 / nan
63 / nan
64 / nan
65 / nan
66 / nan
67 / nan
69 / nan
70 / nan
71 / nan
72 / nan
73 / nan
74 / nan
75 / nan
76 / nan
77 / nan
78 / nan
79 / nan
80 / nan
81 / nan
82 / nan
83 / nan
84 / nan
85 / nan
86 / nan
87 / nan
88 / nan
89 / nan
92 / nan
96 / nan
97 / nan
98 / nan
99 / nan
100 / nan
101 / nan
102 / nan
103 / nan
104 / nan
105 / nan
106 / nan
107 / nan
108 / nan
109 / nan
110 / nan
111 / nan
112 / nan
113 / nan
114 / nan
115 / nan
116 / nan
117 / nan
118 / nan
121 / nan
122 / nan
123 / nan
124 / nan
125 / nan
126 / nan
128 / nan
129 / nan
130 / nan
131 / nan
135 / nan
136 / nan
138 / nan
139 / nan
140 / nan
141 / nan
1

1036 / nan
1039 / nan
1040 / nan
1041 / nan
1042 / nan
1043 / nan
1044 / nan
1046 / nan
1047 / nan
1048 / nan
1049 / nan
1050 / nan
1051 / nan
1052 / nan
1053 / nan
1054 / nan
1055 / nan
1056 / nan
1057 / nan
1058 / nan
1059 / nan
1060 / nan
1061 / nan
1062 / nan
1063 / nan
1064 / nan
1065 / nan
1067 / nan
1069 / nan
1070 / nan
1072 / nan
1073 / nan
1074 / nan
1075 / nan
1076 / nan
1077 / nan
1080 / nan
1081 / nan
1082 / nan
1083 / nan
1084 / nan
1085 / nan
1086 / nan
1087 / nan
1088 / nan
1089 / nan
1090 / nan
1091 / nan
1092 / nan
1093 / nan
1094 / nan
1095 / nan
1096 / nan
1097 / nan
1098 / nan
1099 / nan
1100 / nan
1101 / nan
1102 / nan
1103 / nan
1104 / nan
1105 / nan
1106 / nan
1107 / nan
1110 / nan
1111 / nan
1112 / nan
1113 / nan
1114 / nan
1115 / nan
1116 / nan
1117 / nan
1118 / nan
1119 / nan
1120 / nan
1121 / nan
1123 / 1900-01-01 / 1900-01-01
1138 / nan
1139 / nan
1141 / 1958-01-01 / 1958-01-01
1142 / nan
1143 / nan
1144 / nan
1145 / nan
1146 / nan
1147 / nan
1148 / nan
114

1972 / nan
1973 / nan
1974 / nan
1975 / nan
1976 / nan
1977 / nan
1978 / nan
1982 / nan
1983 / nan
1984 / nan
1987 / 1990-01-01 / 1990-01-01
1988 / nan
1989 / nan
1990 / nan
1991 / nan
1992 / nan
1993 / nan
1994 / nan
1995 / nan
1996 / nan
1997 / nan
1998 / nan
1999 / nan
2000 / nan
2001 / nan
2002 / nan
2003 / nan
2004 / nan
2007 / nan
2008 / nan
2009 / nan
2010 / nan
2011 / nan
2012 / nan
2013 / nan
2014 / nan
2015 / nan
2018 / nan
2019 / 1983-12-31 / 1983-12-31
2021 / nan
2025 / nan
2026 / nan
2027 / nan
2028 / nan
2029 / nan
2030 / nan
2031 / nan
2032 / nan
2033 / nan
2035 / nan
2036 / nan
2037 / nan
2038 / nan
2039 / nan
2042 / nan
2043 / nan
2044 / nan
2045 / nan
2046 / nan
2047 / nan
2048 / nan
2050 / nan
2051 / nan
2052 / nan
2053 / nan
2059 / nan
2061 / nan
2062 / nan
2063 / nan
2064 / nan
2065 / nan
2066 / nan
2067 / nan
2068 / nan
2069 / nan
2070 / nan
2072 / nan
2073 / nan
2074 / nan
2075 / nan
2076 / nan
2077 / nan
2078 / nan
2079 / nan
2080 / nan
2081 / nan
2082 / nan
208

3055 / nan
3056 / nan
3057 / nan
3058 / nan
3059 / nan
3060 / nan
3061 / nan
3062 / nan
3064 / nan
3065 / nan
3067 / 1982-01-01 / 1982-01-01
3068 / nan
3069 / nan
3070 / nan
3072 / nan
3073 / nan
3074 / nan
3075 / nan
3076 / nan
3077 / nan
3078 / nan
3079 / nan
3080 / nan
3081 / nan
3082 / nan
3084 / nan
3088 / nan
3089 / nan
3090 / nan
3091 / nan
3092 / nan
3093 / nan
3094 / nan
3095 / nan
3096 / nan
3097 / nan
3099 / nan
3100 / nan
3101 / nan
3102 / nan
3103 / nan
3104 / nan
3105 / nan
3106 / nan
3107 / nan
3108 / nan
3109 / nan
3110 / nan
3112 / nan
3114 / nan
3115 / nan
3116 / nan
3117 / nan
3118 / nan
3119 / nan
3121 / nan
3122 / nan
3123 / nan
3124 / nan
3125 / nan
3126 / nan
3127 / nan
3128 / nan
3129 / nan
3130 / nan
3131 / nan
3132 / nan
3133 / nan
3134 / nan
3135 / nan
3136 / nan
3137 / nan
3138 / nan
3139 / nan
3140 / nan
3141 / 1992-04-01 / 1992-04-01
3142 / nan
3143 / nan
3145 / nan
3146 / nan
3147 / nan
3148 / nan
3149 / nan
3150 / nan
3151 / nan
3152 / nan
3153 / nan
315

4083 / nan
4084 / nan
4085 / nan
4086 / nan
4087 / nan
4088 / nan
4089 / nan
4090 / nan
4091 / nan
4092 / nan
4093 / nan
4094 / nan
4097 / 1989-01-01 / 1989-01-01
4098 / nan
4099 / nan
4100 / nan
4102 / nan
4103 / nan
4104 / nan
4106 / nan
4107 / nan
4108 / nan
4113 / nan
4115 / nan
4116 / nan
4122 / nan
4123 / nan
4124 / nan
4126 / nan
4127 / nan
4128 / nan
4131 / nan
4132 / nan
4134 / nan
4136 / nan
4137 / nan
4138 / nan
4139 / nan
4140 / nan
4141 / nan
4142 / nan
4145 / 1995-09-01 / 1995-09-01
4146 / nan
4150 / nan
4151 / nan
4153 / nan
4155 / nan
4156 / nan
4160 / nan
4161 / nan
4162 / nan
4164 / nan
4165 / nan
4166 / nan
4167 / nan
4168 / nan
4169 / nan
4171 / nan
4172 / nan
4175 / nan
4176 / nan
4179 / nan
4180 / nan
4181 / nan
4182 / nan
4183 / nan
4184 / nan
4185 / nan
4186 / nan
4188 / nan
4189 / nan
4191 / nan
4192 / nan
4193 / nan
4194 / nan
4196 / nan
4197 / nan
4199 / 1990-06-05 / 1990-06-05
4204 / nan
4205 / nan
4206 / nan
4207 / nan
4210 / nan
4211 / nan
4212 / nan
4213 

...

7980 / 1995-12-25 / 1995-12-25
7981 / 1994-12-25 / 1994-12-25
7983 / 1994-12-25 / 1994-12-25
7984 / 1995-12-25 / 1995-12-25
7985 / 1994-12-25 / 1994-12-25
7986 / 1994-12-25 / 1994-12-25
7987 / 1994-12-25 / 1994-12-25
7988 / 1994-12-25 / 1994-12-25
7989 / 1994-12-25 / 1994-12-25
7990 / 1994-12-25 / 1994-12-25
7991 / 1994-12-25 / 1994-12-25
7992 / 1997-12-25 / 1997-12-25
7993 / 1994-12-25 / 1994-12-25
7994 / 1994-12-25 / 1994-12-25
7995 / 1995-12-25 / 1995-12-25
7996 / 1994-12-25 / 1994-12-25
7997 / 1994-12-25 / 1994-12-25
7998 / 1994-12-25 / 1994-12-25
7999 / 1994-12-25 / 1994-12-25
8000 / 1994-12-25 / 1994-12-25
8001 / 1994-12-25 / 1994-12-25
8002 / 1994-12-25 / 1994-12-25
8003 / 1994-12-25 / 1994-12-25
8004 / 1994-12-25 / 1994-12-25
8005 / 1994-12-25 / 1994-12-25
8006 / 1994-12-25 / 1994-12-25
8007 / 1994-12-25 / 1994-12-25
8008 / 1994-12-25 / 1994-12-25
8009 / 1994-12-25 / 1994-12-25
8010 / 1994-12-25 / 1994-12-25
8011 / 1994-12-25 / 1994-12-25
8012 / 1994-12-25 / 1994-12-25
8013 / 1

8349 / 1994-12-25 / 1994-12-25
8350 / 1994-12-25 / 1994-12-25
8351 / 1994-12-25 / 1994-12-25
8353 / 1994-12-25 / 1994-12-25
8354 / 1994-12-25 / 1994-12-25
8355 / 1994-12-25 / 1994-12-25
8356 / 1994-12-25 / 1994-12-25
8357 / nan
8358 / 1994-12-25 / 1994-12-25
8359 / 1994-12-25 / 1994-12-25
8360 / 1994-12-25 / 1994-12-25
8361 / 1994-12-25 / 1994-12-25
8362 / 1994-12-25 / 1994-12-25
8363 / 1994-12-25 / 1994-12-25
8364 / 1994-12-25 / 1994-12-25
8365 / 1994-12-25 / 1994-12-25
8366 / 1994-12-25 / 1994-12-25
8367 / 1994-12-25 / 1994-12-25
8368 / 1994-12-25 / 1994-12-25
8369 / 1994-12-25 / 1994-12-25
8370 / 1994-12-25 / 1994-12-25
8371 / 1994-12-25 / 1994-12-25
8372 / 1994-12-25 / 1994-12-25
8373 / 1994-12-25 / 1994-12-25
8374 / 1994-12-25 / 1994-12-25
8375 / 1994-12-25 / 1994-12-25
8376 / 1994-12-25 / 1994-12-25
8377 / 1994-12-25 / 1994-12-25
8378 / 1994-12-25 / 1994-12-25
8379 / 1994-12-25 / 1994-12-25
8380 / 1994-12-25 / 1994-12-25
8381 / 1994-12-25 / 1994-12-25
8382 / 1994-12-25 / 1994-12-

8716 / 1994-12-25 / 1994-12-25
8717 / 1994-12-25 / 1994-12-25
8718 / 1994-12-25 / 1994-12-25
8719 / 1994-12-25 / 1994-12-25
8720 / 1994-12-25 / 1994-12-25
8721 / 1994-12-25 / 1994-12-25
8722 / 1994-12-25 / 1994-12-25
8723 / 1994-12-25 / 1994-12-25
8724 / 1994-12-25 / 1994-12-25
8725 / 1994-12-25 / 1994-12-25
8726 / 1994-12-25 / 1994-12-25
8727 / 1994-12-25 / 1994-12-25
8728 / 1994-12-25 / 1994-12-25
8729 / 1994-12-25 / 1994-12-25
8730 / 1994-12-25 / 1994-12-25
8731 / 1994-12-25 / 1994-12-25
8732 / 1994-12-25 / 1994-12-25
8733 / 1994-12-25 / 1994-12-25
8734 / 1994-12-25 / 1994-12-25
8735 / 1994-12-25 / 1994-12-25
8736 / 1994-12-25 / 1994-12-25
8737 / 1994-12-25 / 1994-12-25
8738 / 1994-12-25 / 1994-12-25
8739 / 1994-12-25 / 1994-12-25
8740 / 1994-12-25 / 1994-12-25
8741 / 1994-12-25 / 1994-12-25
8742 / 1994-12-25 / 1994-12-25
8743 / 1994-12-25 / 1994-12-25
8744 / 1994-12-25 / 1994-12-25
8745 / 1994-12-25 / 1994-12-25
8746 / 1994-12-25 / 1994-12-25
8747 / 1994-12-25 / 1994-12-25
8748 / 1

9261 / 1994-12-25 / 1994-12-25
9262 / 1994-12-25 / 1994-12-25
9263 / 1994-12-25 / 1994-12-25
9264 / 1994-12-25 / 1994-12-25
9265 / 1994-12-25 / 1994-12-25
9266 / 1994-12-25 / 1994-12-25
9268 / 1994-12-25 / 1994-12-25
9269 / 1994-12-25 / 1994-12-25
9270 / 1994-12-25 / 1994-12-25
9271 / 1994-12-25 / 1994-12-25
9272 / 1994-12-25 / 1994-12-25
9273 / 1994-12-25 / 1994-12-25
9274 / 1994-12-25 / 1994-12-25
9275 / 1994-12-25 / 1994-12-25
9276 / 1994-12-25 / 1994-12-25
9277 / 1994-12-25 / 1994-12-25
9278 / 1994-12-25 / 1994-12-25
9279 / 1994-12-25 / 1994-12-25
9281 / 1994-12-25 / 1994-12-25
9284 / 1994-12-25 / 1994-12-25
9285 / 1994-12-25 / 1994-12-25
9292 / 1994-12-25 / 1994-12-25
9293 / 1994-12-25 / 1994-12-25
9296 / 1994-12-25 / 1994-12-25
9297 / 1994-12-25 / 1994-12-25
9298 / 1994-12-25 / 1994-12-25
9299 / 1994-12-25 / 1994-12-25
9300 / 1994-12-25 / 1994-12-25
9301 / 1994-12-25 / 1994-12-25
9302 / 1994-12-25 / 1994-12-25
9303 / 1994-12-25 / 1994-12-25
9304 / 1994-12-25 / 1994-12-25
9305 / 1

9898 / 1994-12-25 / 1994-12-25
9900 / 1994-12-25 / 1994-12-25
9901 / 1994-12-25 / 1994-12-25
9902 / 1994-12-25 / 1994-12-25
9907 / 1994-12-25 / 1994-12-25
9912 / 1994-12-25 / 1994-12-25
9913 / 1994-12-25 / 1994-12-25
9914 / 1994-12-25 / 1994-12-25
9921 / 1994-12-25 / 1994-12-25
9926 / 1994-12-25 / 1994-12-25
9930 / 1994-12-25 / 1994-12-25
9932 / 1994-12-25 / 1994-12-25
9933 / 1994-12-25 / 1994-12-25
9934 / 1994-12-25 / 1994-12-25
9943 / 1994-12-25 / 1994-12-25
9944 / 1994-12-25 / 1994-12-25
9947 / 1995-12-25 / 1995-12-25
9948 / 1994-12-25 / 1994-12-25
9949 / 1994-12-25 / 1994-12-25
9950 / 1994-12-25 / 1994-12-25
9952 / 1994-12-25 / 1994-12-25
9953 / 1994-12-25 / 1994-12-25
9954 / 1994-12-25 / 1994-12-25
9955 / 1994-12-25 / 1994-12-25
9956 / 1994-12-25 / 1994-12-25
9957 / 1994-12-25 / 1994-12-25
9958 / 1994-12-25 / 1994-12-25
9959 / 1994-12-25 / 1994-12-25
9960 / 1994-12-25 / 1994-12-25
9962 / 1994-12-25 / 1994-12-25
9966 / 1995-12-25 / 1995-12-25
9968 / 1994-12-25 / 1994-12-25
9969 / 1

In [153]:
print(count2/count)

0.6327631578947368


In [85]:
df.columns

Index(['Unnamed: 0', 'siren', 'nic', 'siret', 'statutDiffusionEtablissement',
       'dateCreationEtablissement', 'trancheEffectifsEtablissement',
       'anneeEffectifsEtablissement',
       'activitePrincipaleRegistreMetiersEtablissement',
       'dateDernierTraitementEtablissement', 'etablissementSiege',
       'nombrePeriodesEtablissement', 'etatAdministratifUniteLegale',
       'statutDiffusionUniteLegale', 'unitePurgeeUniteLegale',
       'dateCreationUniteLegale', 'categorieJuridiqueUniteLegale',
       'denominationUniteLegale', 'sigleUniteLegale',
       'denominationUsuelle1UniteLegale', 'denominationUsuelle2UniteLegale',
       'denominationUsuelle3UniteLegale', 'sexeUniteLegale', 'nomUniteLegale',
       'nomUsageUniteLegale', 'prenom1UniteLegale', 'prenom2UniteLegale',
       'prenom3UniteLegale', 'prenom4UniteLegale', 'prenomUsuelUniteLegale',
       'pseudonymeUniteLegale', 'activitePrincipaleUniteLegale',
       'nomenclatureActivitePrincipaleUniteLegale',
       'ident

In [89]:
df['caractereEmployeurEtablissement'].notnull().sum()

9687

In [106]:
import math
count = 0
for i in range(len(df)):
    try:
        math.notnan(df['caractereEmployeurEtablissement'][i])
        count += 1
    except:
        pass
print(count)

0


In [107]:
15+8.5

23.5

In [ ]:
df['NOM_COMMUE'] = [-40,-30,-24,-1423]

In [116]:
dd = pd.read_csv("socialsante-revenus-et-pauvrete-monetaire-des-menages.csv",sep=';',error_bad_lines=False)

In [119]:
dd.head()

,ANNEE,CODGEO,Libellé commune ou ARM,Ménages fiscaux,Nombre de personnes dans les ménages fiscaux,Médiane revenu disponible par UC (en euros),Part ménages fiscaux imposés,Taux de pauvreté-Ensemble,Taux de pauvreté-moins de 30 ans,Taux de pauvreté-30 à 39 ans,...,Rapport interdécile,1er décile (en euros),9ème décile (en euros),OBJECTID,Geo Point,surfacekm2,gml_id,Geo Shape,codcom,NOM_COMMUNE
0,2012,86244,Saint-Sauvant,570.0,1320.5,18609.333333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,24,"46.3557293591, 0.0779918699072",59.827243,F183409__24,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[0....",244,Saint-Sauvant
1,2012,86226,Saint-Julien-l'Ars,1023.0,2475.0,22295.678571,71.666667,5.06483,NaN,2.23256,...,2.525469,14041.333333,35460.952381,14,"46.5549548764, 0.505403976321",18.633486,F183409__14,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[0....",226,Saint-Julien-l'Ars
2,2012,86163,Montamisé,1335.0,3511.5,24010.666667,78.019505,4.61736,NaN,3.98649,...,2.799124,14537.407407,40692.000000,31,"46.6286023158, 0.43401521969",32.564824,F183409__31,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[0....",163,Montamisé
3,2012,86139,Lusignan,1105.0,2431.0,18889.523809,57.155797,11.27572,NaN,6.32054,...,2.791180,11320.423077,31597.333333,23,"46.4370500114, 0.11405550517",38.022405,F183409__23,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[0....",139,Lusignan
4,2012,86095,Dissay,1271.0,3236.5,19756.800000,64.330709,9.20748,9.74155,8.54907,...,2.625532,12167.692308,31946.666667,28,"46.6963657591, 0.443067472468",23.598945,F183409__28,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[0....",95,Dissay


In [124]:
dd.isnull().sum()

ANNEE                                            0
CODGEO                                           0
Libellé commune ou ARM                           0
Ménages fiscaux                                  0
Nombre de personnes dans les ménages fiscaux     0
Médiane revenu disponible par UC (en euros)      0
Part ménages fiscaux imposés                    50
Taux de pauvreté-Ensemble                       54
Taux de pauvreté-moins de 30 ans                71
Taux de pauvreté-30 à 39 ans                    65
Taux de pauvreté-40 à 49 ans                    64
Taux de pauvreté-50 à 59 ans                    66
Taux de pauvreté-60 à 74 ans                    66
Taux de pauvreté-75 ans ou plus                 68
Taux de pauvreté-propriétaire                   65
Taux de pauvreté-locataire                      60
Part Revenus disponibles-Ensemble               50
Part Revenus d'activité                         50
dont Part Traitements-salaires-chômage          50
dont Part Revenus non salariés 

In [125]:
len(dd)

84

In [142]:
for val in dd.columns:
    print(val, "\t",len(dd)/dd[val].isnull().sum())

ANNEE 	 inf
CODGEO 	 inf
Libellé commune ou ARM 	 inf
Ménages fiscaux 	 inf
Nombre de personnes dans les ménages fiscaux 	 inf
Médiane revenu disponible par UC (en euros) 	 inf
Part ménages fiscaux imposés 	 1.68
Taux de pauvreté-Ensemble 	 1.5555555555555556
Taux de pauvreté-moins de 30 ans 	 1.1830985915492958
Taux de pauvreté-30 à 39 ans 	 1.2923076923076924
Taux de pauvreté-40 à 49 ans 	 1.3125
Taux de pauvreté-50 à 59 ans 	 1.2727272727272727
Taux de pauvreté-60 à 74 ans 	 1.2727272727272727
Taux de pauvreté-75 ans ou plus 	 1.2352941176470589
Taux de pauvreté-propriétaire 	 1.2923076923076924
Taux de pauvreté-locataire 	 1.4
Part Revenus disponibles-Ensemble 	 1.68
Part Revenus d'activité 	 1.68
dont Part Traitements-salaires-chômage 	 1.68
dont Part Revenus non salariés 	 1.68
Part Pensions-retraites-rentes 	 1.68
Part Revenus du patrimoine 	 1.68
Part Prestations sociales-Ensemble 	 1.68
dont Part Prestations familiales 	 1.68
dont Part Minima sociaux 	 1.68
dont Part Prestatio

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in long_scalars
  
